<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)

In [9]:
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib

to_import = ['ABF_Quality_Control.ipynb',
          'Basic_Ephys.ipynb',
          'Firing_Rate_Gain.ipynb',
          'Simple_ABF_tools.ipynb',
          'analyze_IV.ipynb',
          'membrane_analyzer.ipynb',
          'analyze_rheobase.ipynb',
          'fun_math.ipynb',
          'importing_abfs_from_dropbox.ipynb',
          'inputR_from_Iclamp.ipynb',
          'latencey_analyzer.ipynb',
          'QC_recoding_dataframe.ipynb'
            ]

for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


Cloning into 'EphysLib'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 145 (delta 41), reused 29 (delta 29), pack-reused 94
Receiving objects: 100% (145/145), 161.19 KiB | 1.75 MiB/s, done.
Resolving deltas: 100% (85/85), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
'Import and catalog source data'
'RNF182 Dropbox'
data_source = "https://www.dropbox.com/sh/n9t8p257wnzlijk/AAC9Z36JodisyZjnM3mkJC3Xa?dl=0"
file_loc = get_drobox_folder(data_source, 'my_ephys_data.zip')
clear_output(wait=False)
file_naming_scheme = ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type']
abf_recordings_df,protocol_set = catalogue_recs(file_loc,file_naming_scheme)
print('Protocol_Names:')
_ = [print(p) for p in protocol_set]

Protocol_Names:
IC - Gain - D50pA
IC - Rheobase
IC - Sag - D50pA
VC - MemTest-10ms-160ms
VC - Multi IV - 150ms
IC - R input
VC - 3min GapFree
IC - Latentcy 800pA-1s


In [11]:
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',
           'VC - 3min GapFree',
           'VC - Spontaneous-3min-(MT)']
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s']

abf_recordings_df, _ = purge_wrong_clamp(abf_recordings_df,VC_prot,IC_prot)

In [ ]:
def analysis_iterator(abf_recordings_df,func_dict,arg_dict):
    problem_recs = []
    def init_col_object(df,name): 
        df[name] = None
        df[name] = df[name].astype(object)
        return df

    for file_name in tqdm(abf_recordings_df.index):
        abf = abf_or_name(file_name)
        prot_name = abf.protocol


        # check for keyed protocol
        if prot_name not in func_dict.keys():
            # print('unknown protocol(func): ',  prot_name)
            continue
        if prot_name not in arg_dict.keys():
            # print('unknown protocol(args): ',  prot_name)
            continue


        try:
            analyzer_func = func_dict[prot_name]  # get analyzer from dict
            args_for_analyzer =  [abf] + arg_dict[prot_name] # get args for analyzer from dict
            results = analyzer_func(*args_for_analyzer) # run analyzer
            for k in results.keys():

                # New Col?
                cols = abf_recordings_df.columns
                if k not in cols:
                    abf_recordings_df = init_col_object(abf_recordings_df,k)
                abf_recordings_df.at[file_name,k] = results[k]
        except: 
            print('error on: ' ,file_name)
            problem_recs.append(file_name)

    return abf_recordings_df, problem_recs


spike_args =  {'spike_thresh':20, 'high_dv_thresh': 25,'low_dv_thresh': -5,'window_ms': 2}

func_dict = {'IC - Rheobase': rheobase_analyzer }
arg_dict = {'IC - Rheobase': [spike_args, False, False, False] } # [spike_args, to_plot, verbose, force_singlespike]

func_dict = {'IC - Gain - D20pA': gain_analyzer }
arg_dict = {'IC - Gain - D20pA': [spike_args, 0.8, 0] } # [spike_args, to_plot, verbose, force_singlespike]
func_dict = {'IC - Gain - D50pA': gain_analyzer }
arg_dict = {'IC - Gain - D50pA': [spike_args, 0.8, 0] } # [spike_args, to_plot, verbose, force_singlespike]

func_dict = {'VC - MemTest-10ms-160ms': membrane_analyzer }
arg_dict = {'VC - MemTest-10ms-160ms': [False, False, ['Ra', 'Rm', 'Cm', 'tau',	'Cmq',	'Cmf',	'Cmqf', 'Cm_pc']] } # [to_plot, verbose]

func_dict = {'IC - Latentcy 800pA-1s': latencey_analyzer }
arg_dict = {'IC - Latentcy 800pA-1s': [spike_args, False] } # [spike_args, to_plot]



abf_recordings_df, problem_recs = analysis_iterator(abf_recordings_df,func_dict,arg_dict)
print(problem_recs)

abf_recordings_df.to_csv('test.csv')
files.download('test.csv')

 11%|█         | 25/226 [00:03<00:26,  7.71it/s]

In [ ]:
print(problem_recs)


In [ ]:




file_name = 'my_ephys_data/2022_08_12_RNF182/2022x08x12_RNF182_E4KI_F_P251_s001_c002_CA3xPOS_0008.abf'
abf = abf_or_name(file_name)


prot_name = abf.protocol
analyzer_func = func_dict[prot_name]  # get analyzer from dict
args_for_analyzer =  [abf] + arg_dict[prot_name] # get args for analyzer from dict
results = analyzer_func(*args_for_analyzer) # run analyzer
print(results)
# analysis_iterator(abf_recordings_df,func_dict,arg_dict)
